<a href="https://colab.research.google.com/github/sightes/BIGDATA-UBO2022/blob/main/Pyspark_Streaming_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 
import os 
import findspark 
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'
findspark.init()          

In [ ]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
.master('local')\
.appName('Ejemplo1')\
.config('spark.ui.port',4050).getOrCreate()

In [3]:
from pyspark.sql.types import * 
schema=StructType([StructField('dato',IntegerType(),True)])

In [4]:
!mkdir streaming 

In [5]:
people_df=spark.readStream\
.format('csv').schema(schema)\
.option('header',True).load('streaming')

In [6]:
people_df.isStreaming

True

In [7]:
%%python --bg
import time
import pandas as pd 
import numpy as np
def aleatorio():
  return int(99*np.random.normal(0, 0.1)+1)
for n in range(0,100):
  pd.DataFrame(np.asarray([aleatorio() for i in range(0,100)]).reshape(-1,1),columns=['dato'])\
  .to_csv('/content/streaming/data'+ str(n)+'.csv' ,index=False)
  time.sleep(10)

In [ ]:
result_df=people_df.select('*')
query=(result_df.writeStream.format('json').queryName('selectTable')\
       .option('checkpointLocation','checkpoint')\
       .option('path','results')\
       .start()\
       .awaitTermination())
